Topic Modelling of movie plots

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 78.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy

In [4]:
import pandas as pd
import numpy as np

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
!pip install torchtext

ERROR: Operation cancelled by user


In [15]:
import torch
from torchtext.vocab import GloVe



/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [21]:
from transformers import BertTokenizer, BertModel

In [25]:
from sklearn.decomposition import LatentDirichletAllocation

In [29]:
from sklearn.decomposition import NMF

In [31]:
from sklearn.decomposition import TruncatedSVD

In [3]:
nlp=spacy.load("en_core_web_sm")

In [5]:
df=pd.read_csv("/content/wiki_movie_plots_deduped.csv")
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


Preprocessing

In [7]:
def preprocess(text):
  preprocessed_tokens=[token.lemma_.lower() for token in nlp(text) if not token.is_stop and not token.is_punct and token.is_alpha]
  preprocessed_text=" ".join(preprocessed_tokens)
  return preprocessed_text


In [9]:
plots=[]
for plt in df["Plot"][:100]:
  plots.append(preprocess(plt))
plots[:5]

['bartender work saloon serve drink customer fill stereotypically irish man bucket beer carrie nation follower burst inside assault irish man pull hat eye dump beer head group begin wreck bar smash fixture mirror break cash register bartender spray seltzer water nation face group policeman appear order everybody',
 'moon paint smile face hang park night young couple walk past fence learn railing look moon smile embrace moon smile get big sit bench tree moon view block cause frown scene man fan woman hat moon leave sky perch shoulder well',
 'film minute long compose shot girl sit base altar tomb face hide camera center altar view portal display portrait president abraham lincoln james garfield william mckinley victim assassination second shot run second long assassin kneel foot lady justice',
 'last second consist shot shot set wood winter actor represent vice president theodore roosevelt enthusiastically hurry hillside tree foreground fall right cock rifle man bear sign read photograp

TF-IDF Embedding

In [27]:

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(plots)
feature_names = vectorizer.get_feature_names_out()

Glove Embedding

In [18]:
glove = GloVe(name='6B', dim=100)

.vector_cache/glove.6B.zip: 862MB [02:43, 5.27MB/s]                           
100%|█████████▉| 399999/400000 [00:26<00:00, 15022.64it/s]


In [20]:
def document_to_vector(doc, glove):
    vectors = [glove[word] for word in doc.split() if word in glove.stoi]
    if vectors:
        return torch.mean(torch.stack(vectors), dim=0).numpy()
    else:
        return np.zeros(glove.dim)
X_glove = np.array([document_to_vector(plt, love) for plt in plots])

BERT embeddinhg

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [23]:
def get_bert_embeddings(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy().flatten()

X_bert = [get_bert_embeddings(plt, tokenizer, model) for plt in plots]

LDA

In [28]:

num_topics = 3


lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=1)
lda_topic_matrix = lda_model.fit_transform(X_tfidf)


def print_topics(model, feature_names, num_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]))
        print()

print_topics(lda_model, feature_names)

Topic #0:
charlie wife man woman young ambrose dreamer mabel boone hoax

Topic #1:
jack man home girl find leave uncle young begin picture

Topic #2:
charlie film mabel woman scrooge girl jekyll chaplin return wife



Non Negative Matrix Factorization

In [30]:
nmf_model = NMF(n_components=num_topics, random_state=1)
nmf_topic_matrix = nmf_model.fit_transform(X_tfidf)

print_topics(nmf_model, feature_names)

Topic #0:
charlie mabel wife man ambrose room park friend go chaplin

Topic #1:
jack girl man young woman find film take time home

Topic #2:
scrooge christmas way visit office marley partner jacob nephew ghost

